Eodag basics
===========

We present the two basic features of eodag : search and download.

Let's start by creating a workspace directory where all our files and configuration will live:

In [ ]:
import os

workspace = 'eodag_workspace'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

Next, we consider that we have a configuration file sitting in our workspace called `eodag_conf.yml`, with this minimalist content:

    outputs_prefix: <absolute_path_to_eodag_workspace>
    extract: true
    
The rest of the configuration is the providers credentials given as described in the documentation.

The first step is to initialize the session by creating an `EODataAccessGateway` instance, with the previous configuration file:

In [ ]:
# To have some basic feedback on what eodag is doing, we configure logging to output minimum information
from eodag.utils.logging import setup_logging
setup_logging(verbose=1)

from eodag.api.core import EODataAccessGateway

conf_path = os.path.join(workspace, 'eodag_conf.yml')

dag = EODataAccessGateway(user_conf_file_path=conf_path)

Now let's search for Sentinel 2 L1C products in the South of France:

In [ ]:
from datetime import date

product_type = 'S2_MSI_L1C'
extent = {
    'lonmin': -1.999512,
    'lonmax': 4.570313,
    'latmin': 42.763146,
    'latmax': 46.754917
}

products = dag.search(
    product_type,
    startTimeFromAscendingNode='2018-06-01',
    completionTimeFromAscendingNode=date.today().isoformat(),
    geometry=extent
)

Now that we have found some products, we can store the search result in a geojson file that will be used for later download:

In [ ]:
search_result_file = dag.serialize(products, filename=os.path.join(workspace, 'search_results.geojson'))

We can check the extents of the products before downloading them by using ipyleaflet API:

In [ ]:
import ipyleaflet as ipyl

m = ipyl.Map(center=[43.6, 1.5], zoom=5)
layer = ipyl.GeoJSON(data=products.as_geojson_object(), hover_style={'fillColor': 'yellow'})
m.add_layer(layer)
m

Finally, we can download from the eodag SearchResult object `products`, (or from the GeoJson previously created):

In [ ]:
# Download the first product found
dag.download(products[0])

This concludes the tutorial for the basic usage of eodag.